# Fetch historical data

Python script to fetch historical data from binance using ccxt

In [5]:
!pip install openpyxl

  Created wheel for openpyxl: filename=openpyxl-3.0.0-py2.py3-none-any.whl size=241193 sha256=8bc5909533679c0c1e54d9164a2c38f6848177764b3e3c9e8a5c742ebe9a69e7
  Stored in directory: C:\Users\codeninja\AppData\Local\pip\Cache\wheels\34\ee\6c\1279f7b70ea72432c2cef15dd3d915477cb3771d1618f6b8ef
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-cp37-none-any.whl size=8922 sha256=89a5e1f98f64484b70b26d922b0062a6fe4d338d98bb5c2437ea6a4c2a21d38f
  Stored in directory: C:\Users\codeninja\AppData\Local\pip\Cache\wheels\2a\77\35\0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile


In [33]:
# -*- coding: utf-8 -*-

import os
from pathlib import Path

import sys
import csv

# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')

import ccxt  # noqa: E402


# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("./data/raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)


# -----------------------------------------------------------------------------

scrape_candles_to_csv('btc_usdt_1m.csv', 'binance', 3, 'BTC/USDT', '1m', '2019-10-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/eth_btc_1m.csv', 'binance', 3, 'ETH/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/ltc_btc_1m.csv', 'binance', 3, 'LTC/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/xlm_btc_1m.csv', 'binance', 3, 'XLM/BTC', '1m', '2018-01-01T00:00:00Z', 1000)

1000 BTC/USDT candles in total from 2019-10-11T10:20:00.000Z to 2019-10-12T02:59:00.000Z
2000 BTC/USDT candles in total from 2019-10-10T17:40:00.000Z to 2019-10-12T02:59:00.000Z
3000 BTC/USDT candles in total from 2019-10-10T01:00:00.000Z to 2019-10-12T02:59:00.000Z
4000 BTC/USDT candles in total from 2019-10-09T08:20:00.000Z to 2019-10-12T02:59:00.000Z
5000 BTC/USDT candles in total from 2019-10-08T15:40:00.000Z to 2019-10-12T02:59:00.000Z
6000 BTC/USDT candles in total from 2019-10-07T23:00:00.000Z to 2019-10-12T02:59:00.000Z
7000 BTC/USDT candles in total from 2019-10-07T06:20:00.000Z to 2019-10-12T02:59:00.000Z
8000 BTC/USDT candles in total from 2019-10-06T13:40:00.000Z to 2019-10-12T02:59:00.000Z
9000 BTC/USDT candles in total from 2019-10-05T21:00:00.000Z to 2019-10-12T02:59:00.000Z
10000 BTC/USDT candles in total from 2019-10-05T04:20:00.000Z to 2019-10-12T02:59:00.000Z
11000 BTC/USDT candles in total from 2019-10-04T11:40:00.000Z to 2019-10-12T02:59:00.000Z
12000 BTC/USDT cand